In [ ]:
!pip install transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from datasets import load_dataset

# Load the Bangla subset of the xlsum dataset
xlsum_dataset = load_dataset("csebuetnlp/xlsum", "bengali")

# Define a preprocessing function
def preprocess_function(examples):
    # Concatenate text and summary for training
    return {"input_text": examples["text"], "target_text": examples["summary"]}

# Apply the preprocessing function to the dataset
preprocessed_dataset = xlsum_dataset.map(preprocess_function, remove_columns=['text', 'summary'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Map:   0%|          | 0/8102 [00:00<?, ? examples/s]

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5Tokenizer

# Load the custom Bangla tokenizer
flant5_tokenizer_bangla = T5Tokenizer.from_pretrained("midnightGlow/flant5-bangla-tokenizer")

# Tokenize the data
def tokenize_function(examples):
    model_inputs = flant5_tokenizer_bangla(examples["input_text"], max_length=512, truncation=True, padding="max_length")
    labels = flant5_tokenizer_bangla(examples["target_text"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = preprocessed_dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/8102 [00:00<?, ? examples/s]

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

Map:   0%|          | 0/1012 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_bangla",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_steps=100,
    save_steps=500,
    save_strategy="epoch",

)

# Load the Flan-T5 model
flant5_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Resize the model embeddings to match the tokenizer
flant5_model.resize_token_embeddings(len(flant5_tokenizer_bangla))


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(32101, 512)

In [ ]:
trainer = Seq2SeqTrainer(
    model=flant5_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=flant5_tokenizer_bangla,
)

In [ ]:
trainer.train()
trainer.save_model('./saved_flant5_Bangla_model')
flant5_tokenizer_bangla.save_pretrained('./saved_flant5_Bangla_tokenizer')

Epoch,Training Loss,Validation Loss
1,1.700900,1.482252
2,1.589400,1.447570
3,1.589700,1.442601


('./saved_flant5_Bangla_tokenizer/tokenizer_config.json',
 './saved_flant5_Bangla_tokenizer/special_tokens_map.json',
 './saved_flant5_Bangla_tokenizer/spiece.model',
 './saved_flant5_Bangla_tokenizer/added_tokens.json')

In [ ]:
trainer.evaluate()

{'eval_loss': 1.4426013231277466,
 'eval_runtime': 17.8692,
 'eval_samples_per_second': 56.634,
 'eval_steps_per_second': 7.107,
 'epoch': 3.0}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
trained_model = AutoModelForSeq2SeqLM.from_pretrained('./saved_flant5_Bangla_model')
trained_tokenizer = T5Tokenizer.from_pretrained('./saved_flant5_Bangla_tokenizer')

# Sample Bangla text
bangla_text = "রবীন্দ্রনাথ ঠাকুর রবীন্দ্রনাথ ঠাকুর বাঙালির কাছে বিশেষ একটি নাম। বাংলা সাহিত্যের তিনি একজন উজ্জ্বল নক্ষত্র এবং তাঁর বিশাল সাহিত্য কীর্তির জন্য তিনি বহু বাঙালির রক্তস্রোতে আজও মিশে আছেন। তিনি ছিলেন একাধারে বাঙালি কবি, ঔপন্যাসিক, সঙ্গীতকার, চিত্রশিল্পী, নাট্যকার, ছোটগল্পকার, প্রাবন্ধিক ও দার্শনিক। এক কথায় বহুমুখী প্রতিভার সম্বন্বয় ঘটেছিল তাঁর বর্ণময় দীর্ঘ কর্মজীবনে"
text2= "মারধর এবং বিনা কারণে পুলিশে দেয়ার প্রতিবাদে অবস্থান নিয়েছেন মুকিম তিনি বুধবার সন্ধ্যে থেকে এই অবস্থান নিয়েছেন। মঙ্গলবার রাতে তাকেসহ বাকীদেরকে নির্যাতনের পর পুলিশের হাতে তুলে দেয়া হয়। মুকিম চৌধুরী বিবিসিকে জানিয়েছেন, তাকে মারধর এবং বিনা কারণে পুলিশে দেয়ার ঘটনায় জড়িতদের বিচার না হওয়া পর্যন্ত তিনি সেখানে অবস্থান করবেন।"
# Encode the text using the tokenizer
inputs = trained_tokenizer(text2, return_tensors='pt')

# Generate summary or translation
summary_ids = trained_model.generate(inputs['input_ids'], max_length=50, num_beams=5, early_stopping=True)
summary = trained_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated summary/translation:", summary)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated summary/translation: মেনে এবং বিনা কারণে পুলিশে দেয়ার প্রতিবাদে অবস্থান নিয়েছেন মুকিম তিনি বুধবার সন্ধ্যে থেকে এই অবস্থান নিয়েছেন।


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated summary/translation: মেনে এবং বিনা কারণে পুলিশে দেয়ার প্রতিবাদে অবস্থান নিয়েছেন মুকিম তিনি বুধবার সন্ধ্যে থেকে এই অবস্থান নিয়েছেন।


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write)

In [ ]:
!huggingface-cli repo create flant5_xlsum_bangla --type model

git version 2.34.1
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)

You are about to create midnightGlow/flant5_xlsum_bangla
Proceed? [Y/n] y

Your repo now lives at:
  https://huggingface.co/midnightGlow/flant5_xlsum_bangla

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/midnightGlow/flant5_xlsum_bangla



In [ ]:
from huggingface_hub import HfApi, HfFolder

# Define paths
model_path = './saved_flant5_Bangla_model'
tokenizer_path = './saved_flant5_Bangla_tokenizer'
repository_id = 'midnightGlow/flant5_xlsum_bangla'

# Initialize API
api = HfApi()

# Upload the model
api.upload_folder(
    folder_path=model_path,
    repo_id=repository_id,
    repo_type='model',
    commit_message="Initial model upload"
)

# Upload the tokenizer
api.upload_folder(
    folder_path=tokenizer_path,
    repo_id=repository_id,
    repo_type='model',
    commit_message="Initial tokenizer upload"
)


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/midnightGlow/flant5_xlsum_bangla/commit/8b5d910c04d865028b2e9ccca07a22799ca02465', commit_message='Initial tokenizer upload', commit_description='', oid='8b5d910c04d865028b2e9ccca07a22799ca02465', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
trained_model = AutoModelForSeq2SeqLM.from_pretrained('midnightGlow/flant5-xlsum-bangla-state-domain_model')
trained_tokenizer = T5Tokenizer.from_pretrained('midnightGlow/flant5-bangla-tokenizer')

# Sample Bangla text
bangla_text = """আটক স্বজনদের আইডি কার্ড হাতে উইগুর নারীদের বিক্ষোভ (ফাইল ফটো) অভিযোগ করেন জাতিসংঘের জাতিগত বৈষম্য বিষয়ক কমিটির সদস্য গে ম্যাকডুগাল। তিনি বলেন, এমন খবর তিনি পেয়েছেন যে চীনা কর্তৃপক্ষ স্বায়ত্তশাসিত উইগুর প্রদেশকে কার্যত "বিশাল একটি বন্দীশিবিরে" রূপান্তরিত করেছে। চীন সাথে সাথে এর কোনো জবাব দেয়নি। ৫০-সদস্যের চীনা প্রতিনিধিদলের পক্ষ থেকে বলা হয়, সোমবার এই অভিযোগের জবাব দেওয়া হবে। তবে এর আগে বিভিন্ন সময় চীন বলেছে, এ ধরণের বন্দী শিবিরের কোনো অস্তিত্ব নেই। বাজারে উইগুর পুরুষদের জটলা (ফাইল ফটো, ২০১৩)। চীনা কোনো বন্দীশিবিরের অস্তিত্ব অস্বীকার করে। উইগুর মুসলিম কারা? চীনের জিনজিয়াং প্রদেশের জনসংখ্যার ৪৫ শতাংশ উইগুর মুসলিম। এই প্রদেশটি তিব্বতের মত স্বশাসিত একটি অঞ্চল। বিদেশী মিডিয়ার ওপর এখানে যাবার ব্যাপারে কঠোর বিধিনিষেধ রয়েছে। কিন্তু গত বেশ ক'মাস ধরে বিভিন্ন সূত্রে খবরাখবর বেরুচ্ছে যে শিনজিয়াং-এ উইগুর এবং অন্যান্য মুসলিমরা ব্যাপকহারে আটকের শিকার হচ্ছে। ঠিক কি জন্য চীনকে অভিযুক্ত করা হচ্ছে? হিউম্যান রাইটস ওয়াচ এবং অ্যামনেস্টি ইন্টারন্যাশনাল সহ অন্যান্য আন্তর্জাতিক মানবাধিকার সংস্থা জাতিসংঘের মানবাধিকার বিষয়ক কমিটির কাছে দেওয়া তাদের রিপোর্টে বলেছে, উইগুর মুসলিমদের গণহারে ধরে বিভিন্ন বন্দীশিবিরে নেয়া হচ্ছে। এরপর সেসব শিবিরে তাদেরকে জোর করে চীনা প্রেসিডেন্ট শি জিনপিংয়ের প্রতি আনুগত্য প্রকাশে বাধ্য করা হচ্ছে। নির্বাসিত উইগুর মুসলিমদের সংগঠন ওয়ার্ল্ড উইগুর কংগ্রেস তাদের এক রিপোর্টে বলেছে, কোনো অভিযোগ ছাড়াই উইগুরদের ধরে অনির্দিষ্টকালের জন্য আটকে রাখা হচ্ছে, এবং তাদেরকে জোর করে চীনা কম্যুনিস্ট পার্টির পক্ষে স্লোগান দিতে বলা হচ্ছে। ওয়ার্ল্ড উইগুর কংগ্রেস দাবি করছে, বন্দীদের ঠিকমত খাবার দেওয়া হয়না এবং নানাভাবে নির্যাতন করা হয়। বলা হচ্ছে, অধিকাংশ বন্দীকে দীর্ঘদিন আটকে রাখলেও তাদেরকে অভিযুক্ত করা হয়না এবং কোনো আইনি সহায়তা নিতেও দেওয়া হয়না। ধর্মীয় কট্টরবাদ মোকাবেলার যুক্তিতে চীনা কর্তৃপক্ষ এভাবে গণ-আটকের পথ নিয়েছে। আরও পড়ুন: চীনে মসজিদ ভাঙ্গার বিরুদ্ধে দাঁড়িয়েছে মুসলিমরা সামরিক শক্তিতে মার্কিন ভাবনারও সীমা ছাড়িয়েছে চীন তুরস্কের ইস্তাম্বুলে চীনা কনস্যুলেটের বাইরে নির্বাসিত উইগুরদের বিক্ষোভ (ফাইল ফটো) চীন কি বলে? চীনের সরকার সবসময় বলে এ ধরনের কোনো বন্দীশিবিরের কোনো অস্তিত্বই সেখানে নেই। এপ্রিলে মার্কিন একজন সিনিয়র কূটনীতিক ল্যরা স্টোন বলেছিলেন, "পুনঃ-শিক্ষা কেন্দ্রগুলোতে" লাখ লাখ মানুষকে আটকে রাখা হয়েছে। প্রতিক্রিয়ায় চীনের পররাষ্ট্র মন্ত্রণালয়ের মুখপাত্র তখন বলেছিলেন, "সবাই জানে এবং দেখে যে শিনজিয়াংয়ে সব জাতিগোষ্ঠী শান্তি এবং সমৃদ্ধির মাঝে পাশাপাশি বসবাস করছে।" পর্যবেক্ষকরা বলছেন, চীনা মুসলিম সমাজে মধ্যপ্রাচ্যের ঘরানার ইসলামী ভাবধারা ঢুকে পড়ছে বলে বেইজিংয়ের মধ্যে বেশ কিছুদিন ধরে উদ্বেগ শুরু হয়েছে। গতকাল (শুক্রবার) উত্তর-পশ্চিমের নিংশিয়া প্রদেশের উইজু শহরে কর্তৃপক্ষ একটি "অননুমোদিত" মসজিদ ভেঙ্গে ফেলার উদ্যোগ নিলে এলাকার শত শত মুসলিম তাতে বাধা দেয়। চীনা মুসলিমরা শত শত বছর ধরে মসজিদ বা অন্যান্য ধর্মীয় স্থাপনা নির্মাণের সময় চীনের ঐতিহ্যবাহী নকশা এবং স্থাপত্য অনুসরণ করে আসছে। কিন্তু উইজুতে যে মসজিদ নিয়ে গণ্ডগোল বেঁধেছে সেটিতে মধ্যপ্রাচ্যের মত মিনার এবং গম্বুজ তৈরি করা হয়। কর্তৃপক্ষ বলছে এই নকশার কোনো অনুমোদন দেওয়া হয়নি।"""
text2= """ডিজিটাল নিরাপত্তা আইন বাতিলের দাবিতে প্রতিবাদ বিক্ষোভ চলছে। ডিজিটাল নিরাপত্তা আইনের বিভিন্ন ধারায় হয়রানির সুযোগ আছে উল্লেখ করে সাংবাদিক, সুশীল সমাজ এবং আন্তর্জাতিক সংস্থার পক্ষ থেকেও শুরু থেকেই আইনটি নিয়ে আপত্তি ছিল। দেখা যাচ্ছে, আইনটি কার্যকর হওয়ার পর মাত্র দুই বছরেই শত শত মামলায় বহু মানুষকে জেল খাটতে হয়েছে। কারাগারে মারা যাওয়া মুশতাক আহমেদের সঙ্গে একই মামলায় গ্রেপ্তার হয়ে প্রায় পাঁচ মাস কারাবন্দী ছিলেন মোঃ দিদারুল ইসলাম। জামিনে মুক্ত মি. ইসলাম রাষ্ট্রচিন্তা নামের একটি সংগঠনের সদস্য। ডিজিটাল নিরাপত্তা আইন প্রসঙ্গে তিনি বলেন, "আমরা তো আমাদের জীবন দিয়ে জানি। প্রত্যেকটা শব্দ উচ্চারণ করার আগে আমরা দশবার চিন্তা করি। এই শব্দটাতে আমি কি ধরা খাব কিনা! আমার পরিবার, বন্ধু-বান্ধব তারাতো আমার নিরাপত্তা নিয়ে খুবই উদ্বিগ্ন।" পাঁচ মাস জেল খেটেছেন দিদারুল ইসলাম মি. ইসলাম জামিনে মুক্ত হলেও এখনো নানাভাবে প্রশাসনের নজরদারির মধ্যে আছেন বলেই জানান। তার কথায়, সরকারের সমালোচকদের কণ্ঠরোধ আর ভয় দেখানোর এক হাতিয়ারে পরিণত হয়েছে ডিজিটাল নিরাপত্তা আইন। "কথা বলার অধিকার আসলে সবচেয়ে গুরুত্বপূর্ণ অধিকার। এই অধিকারটাকে একেবারে পরিকল্পনা করে আপনার কাছ থেকে নিয়ে নেয়া হয়েছে এবং সবচেয়ে কঠিনভাবে। কত কঠিন সেটাতো আমরা মুশতাকের মৃত্যুর মধ্যে দিয়ে দেখলাম।" আরো পড়ুন: 'হয়রানি' নিয়ে ব্যাপক উদ্বেগ বাংলাদেশে ডিজিটাল নিরাপত্তা আইনে গত দুই বছরে সাংবাদিক, রাজনীতিক, শিল্পী, অভিনেতা, চলচ্চিত্র পরিচালক, গার্মেন্টসকর্মী থেকে শিক্ষক ছাত্র পর্যন্ত আসামী হয়ে জেল খেটেছেন। এ আইন নিয়ে বিতর্ক এবং উদ্বেগের মূল কারণ হিসেবে বলা হয় বেশকিছু ধারার মাধ্যমে যথেচ্ছা হয়রানির সুযোগ রয়েছে। এ আইনের ৪৩ ধারায় আইন শৃঙ্খলা রক্ষাকারী বাহিনীকে বিনা পরোয়ানায় তল্লাশি, জব্দ এবং আটকের অসীম ক্ষমতা দেয়া হয়েছে। আর আইনে ব্যাক্তি বা রাষ্ট্রের ভাবমূর্তী ক্ষুণ্ন করা, ধর্মীয় অনুভূতিতে আঘাত বা উস্কানি, মানহানিকর তথ্য প্রচার ও প্রকাশ এবং আইন শৃঙ্খলার অবনতির মতো বিষয়গুলোতে বিভিন্ন ধারায় অপরাধ ও শাস্তির বিধান রয়েছে। এসব ক্ষেত্রে ভুল ব্যাখ্যার সুযোগ রয়েছে বলে মনে করেন অনেকে। আইনের ৯টি ধারা নিয়ে আপত্তি ছিল সম্পাদক পরিষদের ঢাকা বিশ্ববিদ্যালয়ের আইন বিভাগের অধ্যাপক ড. শাহনাজ হুদা বলেন, সংবিধানে নাগরিকদের স্বাধীন মত প্রকাশের অধিকার দিয়েছে। যেখানে ডিজিটাল নিরাপত্তা আইনে অনেক ক্ষেত্রে মানুষের মতপ্রকাশের স্বাধীনতা খর্ব হচ্ছে। "সবার মধ্যে একটা ভীতি যে এইটা বললে কী হবে! এবং আমরাও বলি যে এতকিছু বলো না তোমার বিপদ হবে। এটা স্বাধীন দেশে আমরা কেন করবো? এটা কিন্তু বেশ স্বার্থকভাবে সরকার করে ফেলেছে। সেল্ফ সেন্সরশিপ একটা ভীতি প্রদর্শন, ভীতি মানুষের মধ্যে ঢুকিয়ে দেয়া।" ১৪টি ধারায় সংঘটিত অপরাধ অজামিনযোগ্য এদিকে শুরু থেকেই ডিজিটাল নিরাপত্তা আইনের ৯টি ধারায় আপত্তি দিয়েছিল সম্মাদক পরিষদ। আইনের ১৪টি ধারায় সংঘটিত অপরাধকে অজামিনযোগ্য হিসেবে রাখা হয়েছে। মুশতাক আহমেদ এবং অন্যান্য আসামীদের বিরুদ্ধে ডিজিটাল নিরাপত্তা আইনের ২১/২৫(১)(খ)৩১/৩৫ ধারায় মামলা দেয়া হয়। এর মধ্যে ২১ ধারার অপরাধ ডিজিটাল নিরাপত্তা আইনে অজামিনযোগ্য। মামলার এজাহারে বলা হয়, মুশতাক আহমেদ এবং আসামীরা সামাজিক যোগাযোগ মাধ্যমে বিভিন্ন গুজব, সরকার বিরোধী পোস্টের মাধ্যমে সমাজে বিভ্রান্তি, অস্থিরতা এবং বিশৃঙ্খলা সৃষ্টির অপরাধ করেছিলেন। শেখ হাসিনা বলেছেন, আইন তার নিজস্ব গতিতে চলছে কারাগারে মুশতাক আহমেদের মৃত্যুতে প্রতিবাদ বিক্ষোভ এবং আন্দোলনের প্রেক্ষিতে ২৭শে ফেব্রুয়ারি প্রধানমন্ত্রীর সংবাদ সম্মেলনে আইনের প্রসঙ্গটি উঠেছিল। সাংবাদিকদের প্রশ্নের জবাবে শেখ হাসিনা তার বক্তব্য স্পষ্ট করেন। প্রধানমন্ত্রী বলেন, "আইনের অপপ্রয়োগ হচ্ছে কিনা এটা হলো দৃষ্টিভঙ্গির ব্যাপার। এখন কোনটা আপনার কাছে অপপ্রয়োগ আর কোনটা অপপ্রয়োগ না এটা একটা আপেক্ষিক ব্যাপার।" "আমি তো মনে করি আইন তার নিজস্ব গতিতে চলছে এবং চলবে। যদি কেউ অপরাধ না করে তার বিচার তার শাস্তি হবে না।" এদিকে দেখা যাচ্ছে ডিজিটাল নিরাপত্তা আইনে মামলা বা গ্রেপ্তার হলে অপরাধ প্রমাণের আগেই দীর্ঘদিন কারাভোগ করতে হচ্ছে। লেখক মুশতাক আহমেদ ছয়বার জামিনের আবেদন করেছিলেন, কিন্তু তাকে জামিন দেয়া হয়নি। মৃত্যুর আগে প্রায় ১০ মাস ধরে তিনি কারাগারে বন্দী ছিলেন। সার্বিক পরিস্থিতি পর্যালোচনা করে এ আইনে সংস্কারের প্রয়োজনীয়তা দেখছেন অনেকেই। অধ্যাপক শাহনাজ হুদা বলেন, তথ্য প্রযুক্তির যুগে ডিজিটাল নিরাপত্তা নিশ্চিত করতে যুগোপযোগী আইন থাকার প্রয়োজন রয়েছে কিন্তু বর্তমান আইনটি ঢেলে সাজানো দরকার। ২০১৮ সালে জাতীয় সংসদে ডিজিটাল নিরাপত্তা আইন পাশ হয় "এখন যেহেতু দুই বছর পার হবার পর আমরা দেখেছি যে কত ধরনের অনিয়ম অপব্যবহার বা অপপ্রয়োগ হয়েছে। সেটা অবশ্যই যারা এক্সপার্ট আছে তাদের মতামত নিয়ে এটাকে অবশ্যই অবশ্যই পুরোপুরিভাবে, ঢালাওভাবে এটা পরিবর্তন করতে হবে।" এদিকে মুশতাক আহমেদের মৃত্যুর পর এখন ডিজিটাল নিরাপত্তা আইনটি বাতিলের দাবি উঠেছে। যদিও অব্যাহত প্রতিবাদের সংস্কার প্রসঙ্গে আইনমন্ত্রী আনিসুল হক বিবিসিকে বলেছেন, এই আইনে কোনো অপরাধের অভিযোগ এলে পুলিশের তদন্তের আগে কাউকে গ্রেপ্তার করা যাবে না বা তার বিরুদ্ধে মামলা নেয়া যাবে না, এমন একটি ব্যবস্থা নেয়া হচ্ছে। কিন্তু এ ব্যবস্থা বিদ্যমান সংকটের কতটা সমাধান করতে পারবে সে প্রশ্ন থেকেই যায়। ডিজিটাল সিকিউরিটি আইন: অনলাইনে যেসব ভুল করলে ফেঁসে যাবেন"""
# Encode the text using the tokenizer
inputs = trained_tokenizer(text2, return_tensors='pt')

# Generate summary or translation
summary_ids = trained_model.generate(inputs['input_ids'], max_length=50, num_beams=5, early_stopping=True)
summary = trained_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated summary/translation:", summary)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generated summary/translation: ডিজিটাল নিরাপত্তা আইন বাতিলের দাবিতে প্রতিবাদ বিক্ষোভ চলছে। ডিজিটাল নিরাপত্তা আইনের বিভিন্ন ধারায় হয়রানির সুযোগ আছে উল্লেখ করে সাংবাদিক, সুশীল সমাজ এবং আন্তর্জাতিক সংস্থার পক্ষ থেকেও শুরু থেকেই আইনটি নিয়ে আপত্তি ছিল। দেখা যাচ্ছে, আইনটি কার্যকর হওয়ার পর মাত্র দুই বছরেই শত শত মামলায়


In [13]:
!pip install nltk sacrebleu rouge_score rouge


    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define the summaries
generated_summary = summary
reference_summary = "ডিজিটাল নিরাপত্তা আইনে কারাবন্দী লেখম মুশতাক আহমেদের মৃত্যুর পর বাংলাদেশে এ আইন নিয়ে বিতর্ক এবং সমালোচনা অব্যাহত রয়েছে। অনেকেরই অভিযোগ এ আইন অধিকাংশ ক্ষেত্রে হয়রানির এবং অপব্যহারের হাতিয়ার হিসেবে ব্যবহৃত হচ্ছে। আইন কার্যকরের মাত্র দুই বছরের মধ্যেই দাবি উঠেছে আইনটি একেবারে বাতিল করে দেয়ার।"

# Tokenize the summaries
generated_tokens = generated_summary.split()
reference_tokens = reference_summary.split()

# Calculate BLEU score
smoothing_function = SmoothingFunction().method1
bleu_score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing_function)

print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("BLEU Score:", bleu_score)


Generated Summary: ডিজিটাল নিরাপত্তা আইন বাতিলের দাবিতে প্রতিবাদ বিক্ষোভ চলছে। ডিজিটাল নিরাপত্তা আইনের বিভিন্ন ধারায় হয়রানির সুযোগ আছে উল্লেখ করে সাংবাদিক, সুশীল সমাজ এবং আন্তর্জাতিক সংস্থার পক্ষ থেকেও শুরু থেকেই আইনটি নিয়ে আপত্তি ছিল। দেখা যাচ্ছে, আইনটি কার্যকর হওয়ার পর মাত্র দুই বছরেই শত শত মামলায়
Reference Summary: ডিজিটাল নিরাপত্তা আইনে কারাবন্দী লেখম মুশতাক আহমেদের মৃত্যুর পর বাংলাদেশে এ আইন নিয়ে বিতর্ক এবং সমালোচনা অব্যাহত রয়েছে। অনেকেরই অভিযোগ এ আইন অধিকাংশ ক্ষেত্রে হয়রানির এবং অপব্যহারের হাতিয়ার হিসেবে ব্যবহৃত হচ্ছে। আইন কার্যকরের মাত্র দুই বছরের মধ্যেই দাবি উঠেছে আইনটি একেবারে বাতিল করে দেয়ার।
BLEU Score: 0.016120076571734802


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge
from evaluate import load

# Load ROUGE metric
rouge = load("rouge")

# Define the summaries
generated_summary = summary
reference_summary = "ডিজিটাল নিরাপত্তা আইনে কারাবন্দী লেখম মুশতাক আহমেদের মৃত্যুর পর বাংলাদেশে এ আইন নিয়ে বিতর্ক এবং সমালোচনা অব্যাহত রয়েছে। অনেকেরই অভিযোগ এ আইন অধিকাংশ ক্ষেত্রে হয়রানির এবং অপব্যহারের হাতিয়ার হিসেবে ব্যবহৃত হচ্ছে। আইন কার্যকরের মাত্র দুই বছরের মধ্যেই দাবি উঠেছে আইনটি একেবারে বাতিল করে দেয়ার।"

# Initialize the ROUGE scorer
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
# Extract the F1-scores for ROUGE-1, ROUGE-2, and ROUGE-L
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']


# Calculate the ROUGE scores
print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("ROUGE scores:", scores)
print("ROUGE-1 F1 Score:", rouge_1_f1)
print("ROUGE-2 F1 Score:", rouge_2_f1)
print("ROUGE-L F1 Score:", rouge_l_f1)
print("BLEU Score:", bleu_score)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'rouge'

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load the pre-trained BART model and tokenizer
model_name = 'facebook/bart-large-cnn'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Define the summaries
generated_summary = summary
reference_summary = "ডিজিটাল নিরাপত্তা আইনে কারাবন্দী লেখম মুশতাক আহমেদের মৃত্যুর পর বাংলাদেশে এ আইন নিয়ে বিতর্ক এবং সমালোচনা অব্যাহত রয়েছে। অনেকেরই অভিযোগ এ আইন অধিকাংশ ক্ষেত্রে হয়রানির এবং অপব্যহারের হাতিয়ার হিসেবে ব্যবহৃত হচ্ছে। আইন কার্যকরের মাত্র দুই বছরের মধ্যেই দাবি উঠেছে আইনটি একেবারে বাতিল করে দেয়ার।"

# Tokenize the inputs
def encode(text):
    return tokenizer(text, return_tensors='pt', max_length=1024, truncation=True, padding=True)

encoded_generated = encode(generated_summary)
encoded_reference = encode(reference_summary)

# Generate the BARTScore
def calculate_bartscore(model, tokenizer, src, tgt):
    with torch.no_grad():
        # Forward pass for the source to target
        output = model(**src, labels=tgt['input_ids'])
        # Calculate the log likelihood of tgt given src
        log_likelihood = output.loss.item()
    return log_likelihood

# Calculate BARTScore from generated to reference
bartscore_generated_to_reference = calculate_bartscore(model, tokenizer, encoded_generated, encoded_reference)
# Calculate BARTScore from reference to generated
bartscore_reference_to_generated = calculate_bartscore(model, tokenizer, encoded_reference, encoded_generated)

# Average the two scores for the final BARTScore
average_bartscore = (bartscore_generated_to_reference + bartscore_reference_to_generated) / 2

print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("BARTScore Generated to Reference:", bartscore_generated_to_reference)
print("BARTScore Reference to Generated:", bartscore_reference_to_generated)
print("Average BARTScore:", average_bartscore)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generated Summary: বাংলাদেশে ডিজিটাল নিরাপত্তা আইন বাতিলের দাবিতে প্রতিবাদ বিক্ষোভ চলছে।
Reference Summary: ডিজিটাল নিরাপত্তা আইনে কারাবন্দী লেখম মুশতাক আহমেদের মৃত্যুর পর বাংলাদেশে এ আইন নিয়ে বিতর্ক এবং সমালোচনা অব্যাহত রয়েছে। অনেকেরই অভিযোগ এ আইন অধিকাংশ ক্ষেত্রে হয়রানির এবং অপব্যহারের হাতিয়ার হিসেবে ব্যবহৃত হচ্ছে। আইন কার্যকরের মাত্র দুই বছরের মধ্যেই দাবি উঠেছে আইনটি একেবারে বাতিল করে দেয়ার।
BARTScore Generated to Reference: 3.079413414001465
BARTScore Reference to Generated: 2.5815343856811523
Average BARTScore: 2.8304738998413086


In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge

# Define the summaries
generated_summary = 'করোনাভাইরাস প্রথম এসেছে?'
reference_summary = "বিজ্ঞানীরা প্রথম নতুন সার্স-কোভিড-টু করোনাভাইরাসের কথা শুনেছিলেন প্রায় এক বছর আগে চীনে এটি ধরা পড়ার পর।"

# Initialize the ROUGE scorer
rouge = Rouge()

# Calculate the ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
# Extract the F1-scores for ROUGE-1, ROUGE-2, and ROUGE-L
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']


# Calculate the ROUGE scores
print("Generated Summary:", generated_summary)
print("Reference Summary:", reference_summary)
print("ROUGE scores:", scores)
print("ROUGE-1 F1 Score:", rouge_1_f1)
print("ROUGE-2 F1 Score:", rouge_2_f1)
print("ROUGE-L F1 Score:", rouge_l_f1)
print("BLEU Score:", bleu_score)

ModuleNotFoundError: No module named 'rouge'

In [2]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
trained_model = AutoModelForSeq2SeqLM.from_pretrained('midnightGlow/flant5_xlsum_bangla')
trained_tokenizer = T5Tokenizer.from_pretrained('midnightGlow/flant5-bangla-tokenizer')

# Sample Bangla text
bangla_text = """যুক্তরাষ্ট্রেই কোভিড-১৯ সংক্রমণের সংখ্যা সবচেয়ে বেশি কিন্তু যে ভাইরাসটি বিশ্ব মহামারির জন্য দায়ী, সেটি হয়তো আমাদের সঙ্গে ছিল আরও আগে থেকে। নতুন এক গবেষণায় সেরকম ইঙ্গিতই পাওয়া যাচ্ছে। যুক্তরাষ্ট্রের সেন্টার ফর ডিজিজ কন্ট্রোল এন্ড প্রিভেনশন (সিডিসি) এই গবেষণা প্রকাশ করেছে 'ক্লিনিক্যাল ইনফেকশাস ডিজিজ' নামের একটি জার্নালে। সরকারি হিসেবে করোনাভাইরাস মহামারি শুরু হয়েছিল গত বছরের ৩১শে ডিসেম্বর, যখন চীনের উহান নগরীর স্বাস্থ্য কর্তৃপক্ষ শ্বাসযন্ত্রে সংক্রমণ ঘটায় এমন এক ভাইরাসের ব্যাপারে সতর্কবার্তা জারি করে। ধারাবাহিকভাবে বহু মানুষ তখন এই রহস্যজনক সংক্রমণের শিকার হচ্ছিল। কিন্তু এখন ১১ মাস পরে এসে গবেষকরা দেখছেন, যুক্তরাষ্ট্রের তিনটি রাজ্যের ৩৯ জন মানুষের শরীরে করোনাভাইরাসের অ্যান্টিবডি তৈরি হয়েছিল চীনে এই ভাইরাসে কথা জানা যাওয়ারও দুই সপ্তাহ আগে। যুক্তরাষ্ট্রে সার্স-কোভিড-টু ভাইরাসের প্রথম সংক্রমণ এবছরের ২১শে জানুয়ারির আগে ধরা পড়েনি। যুক্তরাষ্ট্রের বিভিন্ন স্থান থেকে রেডক্রসের সংগ্রহ করা ৭ হাজার মানুষের রক্তের নমুনা গবেষকরা পরীক্ষা করেছেন কিন্তু সিডিসির এই গবেষণায় দেখা যাচ্ছে, যুক্তরাষ্ট্রে গত বছরের ১৩ই ডিসেম্বর হতে এবছরের ১৭ই জানুয়ারি পর্যন্ত ব্লাড ব্যাংকে নিয়মিত রক্ত দিয়েছেন এমন ৭ হাজার ৩৮৯ জনের রক্তের মধ্যে ১০৬ জনের নমুনায় করোনাভাইরাসের অ্যান্টিবডি রয়েছে। কোন ব্যক্তির রক্তে অ্যান্টিবডি থাকার মানে হচ্ছে তিনি একটি ভাইরাসে আক্রান্ত হয়েছিলেন এবং সেই ভাইরাসের সঙ্গে লড়াই করতে শরীরের রোগ প্রতিরোধ ব্যবস্থা চেষ্টা করেছিল। এই ১০৬টি নমুনার মধ্যে আবার এমন ৩৯ জনের রক্তের নমুনা আছে, যাদের রক্ত নেয়া হয়েছিল গত বছরের ১৩ হতে ১৬ই ডিসেম্বরের মধ্যে। যুক্তরাষ্ট্রের ক্যালিফোর্নিয়া, ওরেগন এবং ওয়াশিংটন রাজ্যে এসব মানুষ রক্ত দেন। তাদের সবার রক্তের নমুনায় পাওয়া গেছে করোনাভাইরাসের অ্যান্টিবডি। এই গবেষণায় আরও দেখা যাচ্ছে, কানেক্টিকাট, আইওয়া, ম্যাসাচুসেটস, মিশিগান, রোড আইল্যান্ড এবং উইসকন্সিন রাজ্য থেকে জানুয়ারির প্রথম দিকে সংগ্রহ করা ৬৭ জনের রক্তের নমুনাতেও আছে করোনাভাইরাসের অ্যান্টিবডি। এটা এসব রাজ্যে করোনাভাইরাস ব্যাপকভাবে ছড়িয়ে পড়ার আগের কথা। যারা এরকম আগে-ভাগে করোনাভাইরাসে আক্রান্ত হয়েছেন তাদের বেশিরভাগই পুরুষ এবং তাদের গড় বয়স ৫২। গবেষণাটি যারা চালিয়েছেন, তাদের বিশ্বাস এর মধ্যে কিছু অ্যান্টিবডি হয়তো বিশ্বে অন্যধরণের যেসব করোনাভাইরাস আগে থেকে ছিল, তার মোকাবেলা করতে গিয়ে তৈরি হয়েছে। কিন্তু তারা একই সঙ্গে একথাও বলছেন, অ্যান্টিবডি পাওয়া মানুষের সংখ্যাটা যেহেতু বেশ বড়, তাই এদের কেউ কেউ সেসময় কোভিড-১৯ দ্বারা আক্রান্ত হয়েছেন বলে ইঙ্গিত পাওয়া যায়। তবে গবেষকরা এখনো মনে করেন যুক্তরাষ্ট্রে মানুষের মধ্যে ব্যাপকভাবে করোনাভাইরাস সংক্রমণ ফেব্রুয়ারি মাসের শেষভাগের আগে শুরু হয়নি। কিন্তু এই গবেষণায় যে তথ্য পাওয়া যাচ্ছে তার ফলে এই মহামারির শুরু সম্পর্কে এতদিন আমরা যা জানতাম, তাতে কি পরিবর্তন আনবে? আরও পড়ুন: করোনাভাইরাস নিয়ে আপনার যা জানা প্রয়োজন করোনাভাইরাস যেভাবে শরীরের ক্ষতি করে 'হার্ড ইমিউনিটি' সম্পর্কে যেসব তথ্য জেনে রাখতে পারেন কখন করোনাভাইরাস প্রথম এসেছে? উহানের এক বন্যপ্রাণীর বাজারের সঙ্গে প্রথম সার্স-কোভিড-টু সংক্রমণের সম্পর্ক আছে বলে সন্দেহ করা হয়। ঠিক কোন মুহূর্তে প্রথম সার্স-কোভিড-টু ভাইরাস এসেছে, সেই প্রশ্নের উত্তর হয়তো আমরা কোনদিনই জানতে পারবো না। ২০১৯ সালের ডিসেম্বরে উহানে প্রথম করোনাভাইরাসের সংক্রমণ ছড়িয়ে পড়ার কথা প্রকাশ পাওয়ার কয়েক সপ্তাহ বা এমনকি কয়েক মাস আগে থেকে এই ভাইরাসটি ছড়াচ্ছিল বলে কয়েকটি ইঙ্গিত পাওয়া যায়। তবে সিডিসির গবেষকরা বলছেন, তাদের গবেষণার একটি সীমাবদ্ধতা হচ্ছে, লোকজন তাদের নিজদেশেই এই ভাইরাসে আক্রান্ত হয়েছে নাকি তারা ভ্রমণের সময় এর শিকার হয়েছে সেটি তারা নির্ধারণ করতে পারছেন না। এই গবেষণার রক্তের নমুনা সংগ্রহ করেছিল রেডক্রস। তারা বলছে, যে রক্তদাতাদের রক্ত তারা সংগ্রহ করেছে, তাদের মধ্যে মাত্র তিন শতাংশ বলেছিল, রক্ত দেয়ার আগের মাসে তারা বিদেশে ভ্রমণে গিয়েছিল। আর যারা বিদেশে গিয়েছিল, তাদের মাত্র পাঁচ শতাংশ বলেছিল তারা এশিয়ায় ভ্রমণ করেছে। অন্য কিছু জায়গায় একই ধরণের গবেষণায় দেখা গেছে, চীনে সরকারিভাবে এই ভাইরাস সম্পর্কে সতর্কতা জারির আগেই সেখানে ভাইরাসের উপস্থিতির প্রমাণ পাওয়া যাচ্ছে। যেমন মে মাসে ফ্রান্সের বিজ্ঞানীরা জানিয়েছিলেন, কিছু নমুনা পরীক্ষা করে দেখা যাচ্ছে ২৭শে ডিসেম্বর প্যারিসের কাছে সন্দেহজনক নিউমোনিয়ার চিকিৎসা দেয়া হচ্ছিল এমন এক ব্যক্তির আসলে করোনাভাইরাস হয়েছিল। কয়েকটি দেশের গবেষকরা সেখানে করোনাভাইরাসের প্রাদুর্ভাবের সরকারি ঘোষণার আগের মাসগুলোতে সংগ্রহ করা পয়নিষ্কাশন ব্যবস্থার বর্জ্য পানির নমুনা পরীক্ষা করেছেন। সেখানে তারা করোনাভাইরাসের উপস্থিতির প্রমাণ পেয়েছেন। জুন মাসে ইতালির বিজ্ঞানীরা বলেন, ১৮ই ডিসেম্বরেই মিলান এবং তুরিনে নর্দমার পানিতে করোনাভাইরাসের উপস্থিতির প্রমাণ দেখা যাচ্ছে। অথচ ইতালিতে প্রথম করোনাভাইরাস ধরা পড়েছে এর অনেক পরে। এদিকে স্পেনেও একটি গবেষণায় দেখা গেছে, বার্সেলোনায় জানুয়ারির মধ্যভাগে যে বর্জ্য পানির নমুনা সংগ্রহ করা হয়, সেখানে করোনাভাইরাস আছে। অথচ বার্সেলোনায় করোনাভাইরাসের প্রথম সংক্রমণ ধরা পড়েছে আরও ৪০ দিন পর। স্পেন এবং ব্রাজিলের গবেষকরা পয়নিষ্কাশন প্রণালীর বর্জ্য পানিতে করোনাভাইরাসের অস্তিত্ব খুঁজে পেয়েছেন। ব্রাজিলেও আসলে করোনাভাইরাস কখন পৌঁছেছিল সেটা নিয়ে প্রশ্ন আছে। ২৬শে ফেব্রুয়ারি সেখানে প্রথম সংক্রমণের ঘটনা পরীক্ষায় ধরা পড়ে। ইতালিতে বেড়াতে গিয়েছিলেন ৬১ বছর বয়স্ক এমন এক ব্যক্তি সংক্রমণের শিকার হয়েছিলেন। ইতালি ততদিনে করোনাভাইরাসের দ্বিতীয় প্রধান কেন্দ্রে পরিণত হয়েছে। কিন্তু ফেডারেল ইউনিভার্সিটি অব সান্তা ক্যাটারিনার (ইউএফএসসি) একদল গবেষক সেখানে ২৭শে নভেম্বরেই বর্জ্য পানিতে করোনাভাইরাসের উপস্থিতির প্রমাণ পেয়েছেন। অসওয়াল্ড ক্রুজ ফাউন্ডেশনের আরেকটি গবেষণাতেও দেখা গেছে, ব্রাজিলে সরকারিভাবে করোনাভাইরাসের সংক্রমণের ঘোষণার এক মাস আগে অন্তত একটি সংক্রমণের কথা জানা যাচ্ছে। ১৯ হতে ২৫শে জানুয়ারির মধ্যে এই সংক্রমণ ঘটেছে। তবে এই সংক্রমণের সঙ্গে বিদেশ ভ্রমণের সম্পর্ক ছিল কিনা, সেটি জানা যায়নি। করোনাভাইরাস প্রথম যখন ব্রাজিলে ধরা পড়ে বলে বলা হয় তারও এক মাস আগে থেকে এট সেখানে ছিল বলে প্রমান আছে তবে যেটা এখনো পরিষ্কার নয় কখন কীভাবে এবং কখন সার্স-কোভিড-টু ভাইরাস লোকজনকে সংক্রমিত করতে শুরু করে। আর এই ভাইরাস প্রাণী থেকে মানুষের শরীরে কখন ঢুকেছিল, সেটাও পরিষ্কার নয়। এ পর্যন্ত সবার মনোযোগ কেবল চীনের উহান নগরীর একটি বাজারের দিকেই নিবদ্ধ ছিল, যেখানে মৃত এবং জীবিত বন্য প্রাণী বিক্রি করা হতো। শুরুর দিকের অনেক করোনাভাইরাসের সংক্রমণের সঙ্গে এই বাজারের সম্পর্ক ছিল বলে দেখা গেছে। কিন্তু গবেষকরা নিশ্চিত নন, ভাইরাসটি সেখান থেকেই এসেছে নাকি সেখানে উপযুক্ত পরিবেশ পেয়ে ভাইরাসটি ব্যাপকভাবে বিস্তার লাভ করেছে এবং একজনের দেহ থেকে আরেকজনের দেহে সংক্রমিত হয়েছে। ইউনিভার্সিটি অব হংকং এর মাইক্রোবায়োলজিস্ট ইউয়েন কোক ইয়াং বলেন, "যদি আমাকে জিজ্ঞেস করেন, কোনটির সম্ভাবনা বেশি, আমি বলবো ভাইরাস সেখান থেকেই আসে যেখানে বন্য প্রাণী বিক্রি করা হয়।" চীন করোনাভাইরাস ছড়িয়ে পড়ার যে সময়টির কথা প্রথমে বলেছিল, সেটি পরে তারা আরও পিছিয়ে দিয়েছে। এটি অস্বাভাবিক কিছু নয়। কোভিড-১৯ এর মতো দ্রুত ছড়াতে থাকা একটি ভাইরাসের বিষয়ে তদন্তে এরকম হতেই পারে। উহানের ডাক্তারদের পরিচালিত এর আগের এক গবেষণা এ বছরের শুরুর দিকে প্রকাশ করা হয় মেডিক্যাল জার্নাল ল্যান্সেটে। এতে বলা হয়েছিল, প্রথম করোনাভাইরাসের সংক্রমণ শনাক্ত হয় ডিসেম্বরের ১ তারিখে, এবং তার সঙ্গে কোন বাজারের কোন সম্পর্ক ছিল বলে মনে হচ্ছিল না। কিছু বিশেষজ্ঞ যুক্তি দিচ্ছেন যে, একটি ভাইরাস, যা থেকে একটি বিশ্ব মহামারি শুরু হতে পারে, সেটি কয়েক মাস ধরে শনাক্ত না হয়ে সারা দুনিয়ায় ছড়িয়ে পড়া কঠিন। তবে এটি কয়েক সপ্তাহ ধরে সবার অগোচরে বিস্তার লাভ করছিল- এমনটাই তাদের কাছে অধিকতর বিশ্বাসযোগ্য মনে হয়। বিশেষ করে, বিশ্বের উত্তর গোলার্ধে শীতের সময়।"""


# Encode the text using the tokenizer
inputs = trained_tokenizer(bangla_text, return_tensors='pt')

# Generate summary or translation
summary_ids = trained_model.generate(inputs['input_ids'], max_length=50, num_beams=5, early_stopping=True)
summary = trained_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated summary/translation:", summary)

Generated summary/translation: করোনাভাইরাস প্রথম এসেছে?
